# IS 445 Homework #5 Jekyll Webpage - Bigfoot Sightings Visualizations

This Homework showcases 2 visualizations of Bigfoot sighting data from the `bfro_reports_fall2022.csv` dataset, created using Altair, Python, & Vega-Lite. The 1st is a bar chart of sightings by state, & the 2nd is an interactive area chart of temperature over time. These are exported as JSON files & embedded for display on the Jekyll webpage.

## Importing Libraries

In [1]:
# Import libraries.
import altair as alt
import datetime as dt
import pandas as pd

## Loading Dataset

In [2]:
# Disable max rows limitation for Altair.
alt.data_transformers.disable_max_rows()

# Load dataset.
df = pd.read_csv("https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/bfro_reports_fall2022.csv")

## Data Transformation For Plot 1

In [3]:
# Data transformation for Plot 1: Count sightings by state.
state_counts = df["state"].value_counts().reset_index()
state_counts.columns = ["state", "count"]

## Plot 1 - Bar Chart of Bigfoot Sightings by State

In [4]:
# Plot 1: Bar Chart of Bigfoot sightings by state.
bar_chart = alt.Chart(state_counts).mark_bar().encode(
    x = alt.X("state:N", title = "State", sort = "-y"),
    y = alt.Y("count:Q", title = "Number of Sightings"),
    color = alt.Color("state:N", legend = None, scale = alt.Scale(scheme = "tableau20")),
    tooltip = ["state", "count"]
).properties(
    width = 600,
    height = 400,
    title = "Bigfoot Sightings by State"
)

# Save Plot 1 to JSON.
bar_chart.save("../assets/json/bar_chart.json")
bar_chart.display()

alt.Chart(...)

## Data Transformation For Plot 2

In [5]:
# Data transformation for Plot 2: Filter & prepare temporal data.
df["date"] = pd.to_datetime(df["date"], errors = "coerce")  # Convert to timestamp.
filtered_data = df.dropna(subset = ["date", "temperature_mid"])  # Drop missing values.

## Plot 2 - Interactive Area Chart of Temperature Over Time

In [6]:
# Plot 2: Interactive Area Chart of temperature over time.
# Define date range for initial brush selection.
date_range = (dt.date(2016, 1, 1), dt.date(2018, 12, 31))

# Define brush selection for interactivity.
brush = alt.selection_interval(encodings = ["x"],
                               value = {"x": date_range})

# Base chart for Plot 2: Area chart of temperature over time.
filtered_data_sample = filtered_data.sample(500, random_state = 42)  # Sample 500 rows.
base = alt.Chart(filtered_data_sample, width = 600, height = 200).mark_area().encode(
    x = "date:T",
    y = "temperature_mid:Q"
)

# Upper Chart: Zoomed-in view based on brush selection.
upper = base.encode(
    alt.X("date:T").scale(domain = brush),
    y = alt.Y("temperature_mid:Q", title = "Temperature (°F)", scale = alt.Scale(zero = False)),
    color = alt.Color("season:N", title = "Season", scale = alt.Scale(scheme = "set3")),
    tooltip = ["date", "temperature_mid", "season"]
).properties(
    title = "Bigfoot Sightings: Temperature Over Time (Detail)"
)

# Lower Chart: Overview with brush.
lower = base.encode(
    y = alt.Y("temperature_mid:Q", title = "Temperature (°F)")
).properties(
    height = 60,
    title = "Select a Time Range Below"
).add_params(brush)

# Combine upper & lower charts vertically.
area_chart = upper & lower

# Save Plot 2 to JSON.
area_chart.save("../assets/json/area_chart.json")
area_chart.display()

alt.VConcatChart(...)